In [60]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY =

In [61]:
# First, import the relevant modules
import requests
import collections
import json
from pandas.io.json import json_normalize

+ 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [62]:
# Call the Quandl API pull the JSON structure for 2017
r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X-Carl-Zeiss-Meditec-AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=fHndW7dcmXhU25tFBg1a")

In [63]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data = r.json()
print(json_data.keys())
print(json_data['dataset_data'].keys())

dict_keys(['dataset_data'])
dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [64]:
#Check relevant data
print(json_data['dataset_data']['column_names'])
print(json_data['dataset_data']['data'][:3])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None], ['2017-12-27', 51.45, 51.89, 50.76, 51.82, None, 57452.0, 2957018.0, None, None, None]]


In [65]:
# Define core2017, a list of lists containing the core data
core2017=json_data['dataset_data']['data']


+ 2. Convert the returned JSON object into a Python dictionary.

In [66]:
# Create a Python dictionary
dict2017 = dict(zip(json_data['dataset_data']['column_names'], list(zip(*core2017))))
dict2017.keys()

dict_keys(['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'])

In [67]:
# Check if there are any inconsistent entries
print(any(not isinstance(element, float) for element in dict2017["Open"]))
print(any(not isinstance(element, float) for element in dict2017["Close"]))
print(any(not isinstance(element, float) for element in dict2017["High"]))
print(any(not isinstance(element, float) for element in dict2017["Low"]))
print(any(not isinstance(element, float) for element in dict2017["Traded Volume"]))

True
False
False
False
False


In [68]:
# Verify that Open column contains some None
print(dict2017["Open"])

(51.76, 51.65, 51.45, 51.05, 51.16, 51.88, 52.73, 52.37, 52.7, 53.11, 52.64, 52.29, 52.28, 51.5, 50.89, 50.8, 51.21, 49.5, 49.52, 48.64, 49.64, 49.09, 49.13, 49.11, 48.8, 48.4, 47.25, 46.57, 47.03, 47.09, 47.98, 48.4, 48.38, 47.3, 47.65, 46.42, 46.16, 45.81, 45.0, 45.88, 46.29, 46.53, 45.48, 45.2, 45.01, 45.16, 44.9, 45.08, 45.72, 46.01, 45.8, 45.61, 45.5, 45.58, 45.97, 45.64, 46.2, 46.19, 46.01, 45.36, 44.51, 43.58, 42.0, 42.35, 42.3, 42.3, 41.48, 42.29, 42.54, 42.65, 42.5, 42.29, 42.35, 42.49, 43.21, 42.81, 42.7, 43.0, 42.66, 43.0, 42.38, 42.16, 42.0, 42.0, 41.71, 42.11, 42.64, 42.72, 42.82, 42.46, 42.42, 42.28, 41.88, 42.4, 42.53, 42.12, 41.3, 41.73, 43.5, 44.9, 45.85, 45.13, 45.34, 45.25, 45.24, 44.94, 45.26, 45.16, 44.91, 44.7, 45.31, 45.57, 45.74, 45.06, 45.5, 45.6, 45.07, 44.67, 44.29, 44.94, 44.64, 44.79, 45.5, 44.67, 45.83, 45.29, 45.01, 45.73, 46.68, 47.23, 46.95, 47.29, 47.03, 47.46, 46.48, 46.9, 45.66, 46.34, 46.52, 46.5, 47.31, 46.77, 47.8, 47.01, 47.12, 46.8, 46.12, 45.22

In [69]:
# Clean Open column by mapping it to 0.0
dict2017["Open"]= tuple(map((lambda x: x if isinstance(x, float) else 0.0), dict2017["Open"]))


+ 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [70]:
# Print the maximum and minimum values
print(max(dict2017["Open"]))
# Exclude 0.0 to find the actual minimum value
print(min(x for x in dict2017["Open"] if x!=0))

53.11
34.0



+ 4. What was the largest change in any one day (based on High and Low price)?

In [71]:
# Same results, two different ways
max(high-dict2017["Low"][index] for index, high in enumerate(dict2017['High']))
max(high-low for high, low in zip(dict2017['High'], dict2017["Low"]))

2.8100000000000023


+ 5. What was the largest change between any two days (based on Closing Price)?

In [72]:
max(abs(dict2017["Close"][index] - dict2017["Close"][index-2]) for index in range(2, len(dict2017["Close"])))

3.1499999999999986


+ 6. What was the average daily trading volume during this year?

In [73]:
# Total sum of the traded volume divided by the number of recorded days
sum(dict2017["Traded Volume"]) / len(dict2017["Traded Volume"])

89124.33725490196

+ 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [74]:
# A function to find the median in a SORTED list
def findMedian(inputlist):
    
    # the length of the list
    length = len(inputlist)
    
    # if the list is empty, i.e. the length is 0, there cannot be a median
    if length == 0: 
        raise ValueError('No median can be found')
        
    # if the length of the list is even, the median is the average of the middle two values in a sorted list
    if length % 2 == 0:
        return (inputlist[(length/2)]+inputlist[(length/2)-1])/2
    
    # if the length of the list is odd, the median is the middle value in a sorted list
    else: 
        return inputlist[int(length/2)]
    
# Sort Traded Volume column
sortedTrade = sorted(dict2017["Traded Volume"])

# Call a function to compute the median
median = findMedian(sortedTrade)

In [75]:
print(median)

76286.0
